### Set-up

In [16]:
%pip uninstall logikon -y

Found existing installation: logikon 0.0.1.dev0
Uninstalling logikon-0.0.1.dev0:
  Successfully uninstalled logikon-0.0.1.dev0
Note: you may need to restart the kernel to use updated packages.


In [2]:
import getpass
import os

GH_ACCESS_TOKEN = getpass.getpass("GH_ACCESS_TOKEN: ")
os.environ["GH_ACCESS_TOKEN"] = GH_ACCESS_TOKEN

In [3]:
# install logikon module from github repo
%pip install hatch
%pip install git+https://$GH_ACCESS_TOKEN@github.com/logikon-ai/logikon.git

Note: you may need to restart the kernel to use updated packages.
  Cloning https://****@github.com/logikon-ai/logikon.git to /tmp/pip-req-build-reee8iei
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/logikon-ai/logikon.git' /tmp/pip-req-build-reee8iei
  Resolved https://****@github.com/logikon-ai/logikon.git to commit 02aca9e66042d28b8e788980297d655bfc30b4b7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for logikon: filename=logikon-0.0.1.dev0-py3-none-any.whl size=24425 sha256=d7fd58db8c323f1fa5fa684374f3efeac5c918624c762ce944085e7bcf7b2505
  Stored in directory: /tmp/pip-ephem-wheel-cache-lmdqy_i9/wheels/10/bc/0f/621fa7ad65eb758157ed6c6ae76ca3429cca25fbbebd263f3f
Successfully built logikon
Note: you may need to restart the kernel to use updated packages.


### Imports and Api Keys

In [4]:
import getpass
import os

import logikon

OPENAI_API_KEY = getpass.getpass("OPENAI_API_KEY: ")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

HF_ACCESS_TOKEN = getpass.getpass("HF_ACCESS_TOKEN: ")
os.environ["HF_ACCESS_TOKEN"] = HF_ACCESS_TOKEN

### Data

In [5]:
# slightly adapted from `logikon/oasst1-delib` dataset

prompt = """prompter:
I live in the US, and i am thinking of moving to Argentina.
What pro and cons you think i will have?

assistant:
"""
completion = """Here are some pros and cons to consider if you are thinking of moving to Argentina:
Pros: 
Rich culture: Argentina has a rich cultural heritage, including music, dance, literature, and art.
Beautiful natural scenery: The country has a varied geography, from the Andes mountains to the fertile plains of the Pampas, and plenty of natural parks and reserves.
Good food: Argentine cuisine is known for its grilled meats and various types of pasta, and there are many great restaurants to be found throughout the country.
Low cost of living: In comparison to the United States, the cost of living in Argentina can be quite low, especially in terms of housing and food.
Cons:
Economic instability: Argentina has a history of economic instability, with periodic devaluations of its currency, high inflation, and government default on its debt.
High crime rates: Some areas of Argentina have high crime rates, particularly in the larger cities, so it's important to be cautious and aware of your surroundings.
Language barrier: Although many people in Argentina speak some English, it's still a good idea to learn Spanish if you plan to live there, as it may be difficult to get by without it in some areas.
Distance from home: Moving to a foreign country can be a major adjustment, especially if you are far from friends and family.
These are some of the factors to consider if you're thinking of moving to Argentina.
Ultimately, the decision to move will depend on your individual circumstances and priorities."""

### Example Usage 1:<br/>Create Informal Argument Map with OpenAI (Default Configuration)

In [6]:
config = logikon.DebugConfig(
    expert_model='gpt-3.5-turbo-instruct',
    expert_model_kwargs={'temperature': 0.7},
    #artifacts=['claims']
)
print(config)

expert_model='gpt-3.5-turbo-instruct' expert_model_kwargs={'temperature': 0.7} llm_framework='OpenAI' generation_kwargs=None inputs=[] metrics=[] artifacts=['informal_argmap'] report_to=[]


In [11]:
results = logikon.score(prompt, completion, config=config)



> Entering new ClaimExtractionChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful, honest and knowledgeable AI assistant with expertise in critical thinking and argumentation analysis. Always answer as helpfully as possible.

# Your Assignment

Identify the basic question addressed in a text.

# Detailed Instructions

Use the inputs (a TEXT) to solve your assignment and read the following instructions carefully.
Identify the overarching question the TEXT raises and addresses.
Note that the overarching question is not necessarily explicitly stated in the TEXT (as shown in some examples below). It may be implicit. And it may deviate from any explicitly stated questions or instructions.
State a single bold question in a concise and most-simple way. Use plain language.
Don't provide alternatives, comments or explanations.

# Example

TEXT:
:::
Should we visit Mars?
- Mars is a planet and rather difficult to reach.
- Visiting Mars would be a great 

In [14]:
results

DebugResults(artifacts=[Artifact(id='claims', description='Key claims in the deliberation', metadata=None, data=['Yes, you should move to Argentina.'], dtype=None), Artifact(id='informal_argmap', description='Informal Argument Map', metadata=None, data={'nodelist': [{'id': '686b37a8-11b1-493c-b1eb-16d8d431f40c', 'text': 'Yes, you should move to Argentina.', 'label': 'Claim-1', 'annotations': [], 'nodeType': 'proposition'}, {'id': '0bebb2ac-549a-4bbb-9544-8cf03218c57f', 'text': 'Argentina has a rich cultural heritage, including music, dance, literature, and art.', 'label': '"Argentina: A Cultural Haven"', 'annotations': [{'start': 71, 'end': 150}], 'nodeType': 'proposition'}, {'id': '77c31771-869c-4fb5-95b1-32466ddc4f52', 'text': 'The country has a varied geography, from the Andes mountains to the fertile plains of the Pampas, and plenty of natural parks and reserves.', 'label': ':::', 'annotations': [{'start': 150, 'end': 286}], 'nodeType': 'proposition'}, {'id': '2ef44412-8939-4aaa-9f

In [15]:
results.artifacts[1].data

{'nodelist': [{'id': '686b37a8-11b1-493c-b1eb-16d8d431f40c',
   'text': 'Yes, you should move to Argentina.',
   'label': 'Claim-1',
   'annotations': [],
   'nodeType': 'proposition'},
  {'id': '0bebb2ac-549a-4bbb-9544-8cf03218c57f',
   'text': 'Argentina has a rich cultural heritage, including music, dance, literature, and art.',
   'label': '"Argentina: A Cultural Haven"',
   'annotations': [{'start': 71, 'end': 150}],
   'nodeType': 'proposition'},
  {'id': '77c31771-869c-4fb5-95b1-32466ddc4f52',
   'text': 'The country has a varied geography, from the Andes mountains to the fertile plains of the Pampas, and plenty of natural parks and reserves.',
   'label': ':::',
   'annotations': [{'start': 150, 'end': 286}],
   'nodeType': 'proposition'},
  {'id': '2ef44412-8939-4aaa-9fc4-82fa79f61ab4',
   'text': 'Argentine cuisine is known for its grilled meats and various types of pasta, and there are many great restaurants to be found throughout the country.',
   'label': '"Argentina: A Fo

In [16]:
len(results.artifacts[1].data["nodelist"])

19